In [166]:
from TextNormalizer import TextNormalizer
from flashtext import KeywordProcessor
import re
import pickle

In [152]:
def save_bin(fname,var):
    with open(fname,'wb') as fp:
        pickle.dump(var,fp)

In [74]:
def load_bin(fname):
    with open(fname,'rb') as fp:
        return pickle.load(fp)

In [86]:
def load_txt(fname,vec_delim = '/',phrase_delim='\n'):
    with open(fname) as f:
        return [list(filter(lambda x: x!='',elem.split(phrase_delim))) for elem in f.read().split(vec_delim)]

In [103]:
def normalize_vecs(vecs):
    return [[' '.join(TextNormalizer.normalize(phrase)) for phrase in vec] for vec in vecs]

In [104]:
def normalize_docs(docs):
    return [' '.join(TextNormalizer.normalize(doc)) for doc in docs]

In [162]:
docs = load_bin('docs.txt')

In [180]:
re.split('\. |! |\?',docs[1])

['<h1>Обслуживание кондиционеров</h1> Какой бы ни была цена на сервисное обслуживание кондиционеров, а заниматься поломанными или вышедшими из строя кондиционерами всё равно рано или поздно нужно',
 'Однако если вы дорожите не только своим аппаратом, но и денежными средствами – закажите техническое обслуживание кондиционеров у нас, ведь мы предлагаем самую низкую стоимость на услуги',
 ' Кстати, если вам пока обслуживать по сути ничего, потому что вы так и не приобрели кондиционер, то вам стоит обратить внимание на наш <h href=“на_группу_1”магазин кондиционеров</a>',
 'Вероятно, после покупки и установки вам потребуется масса других услуг по сервису вашего кондиционера',
 'Итак, к списку услуг по обслуживанию кондиционеров в нашем сервисном центре мы относим: ремонт сплит систем, установка новых сплитов, чистка, замена комплектующих',
 'Кроме того, предлагаем диагностику и регулярную замену труб',
 'Если вас интересует, сколько стоит обслуживание именно вашего кондиционера – обратитесь

In [154]:
nvecs = load_bin('nvecs')
ndocs = load_bin('ndocs')

In [186]:
test1 = [' '.join(TextNormalizer.normalize(sent)) for sent in re.split('\. |! |\?',docs[1])]

In [190]:
test1

['h1 обслуживание кондиционер h1 цена сервисный обслуживание кондиционер заниматься поломать выйти строй кондиционер равно рано поздно',
 'однако дорожить аппарат денежный средство заказать технический обслуживание кондиционер предлагать низкий стоимость услуга',
 'кстати обслуживать суть ничто приобрести кондиционер стоить обратить внимание href на_группу_1 магазин кондиционер',
 'вероятно покупка установка потребоваться масса услуга сервис кондиционер',
 'итак список услуга обслуживание кондиционер сервисный центр относить ремонт сплит система установка новый сплит чистка замена комплектовать',
 'кроме предлагать диагностик регулярный замена труба',
 'интересовать сколько стоить обслуживание именно кондиционер обратиться телефон указанный сайт приходить офис обсудить прайс напрямую мастер',
 'обратить внимание обслуживать кондиционер вне зависимость фирма производитель',
 'чаща обращаться пользователь марка daikin mitsubishi panasonic lg',
 'потенциальный обладатель кондиционер приго

In [209]:
def locate_phrase(phrase, sentence):
    ph_words = phrase.split()
    keyword_processor = KeywordProcessor()
    for word in ph_words:
        keyword_processor.add_keyword(word)
    found_words = ' '.join(keyword_processor.extract_keywords(sentence))
    keyword_processor = KeywordProcessor()
    keyword_processor.add_keyword(phrase)
    for elem in found_words:
        found = keyword_processor.extract_keywords(found_words)
        if found:
            return found
    return []

In [210]:
tt = [locate_phrase('сколько стоить обслуживание кондиционер',sent) for sent in test1]

In [211]:
tt

[[],
 [],
 [],
 [],
 [],
 [],
 ['сколько стоить обслуживание кондиционер'],
 [],
 [],
 []]

In [160]:
locate_phrase('сколько стоить обслуживание кондиционер',ndocs[1])

[('обслуживание', 3, 15),
 ('кондиционер', 16, 27),
 ('обслуживание', 46, 58),
 ('кондиционер', 59, 70),
 ('кондиционер', 103, 114),
 ('обслуживание', 196, 208),
 ('кондиционер', 209, 220),
 ('кондиционер', 297, 308),
 ('стоить', 309, 315),
 ('кондиционер', 359, 370),
 ('кондиционер', 432, 443),
 ('обслуживание', 463, 475),
 ('кондиционер', 476, 487),
 ('сколько', 649, 656),
 ('стоить', 657, 663),
 ('обслуживание', 664, 676),
 ('кондиционер', 684, 695),
 ('кондиционер', 806, 817),
 ('кондиционер', 945, 956),
 ('кондиционер', 1005, 1016)]

In [143]:
def relevance(vec,doc):
    keyword_processor = KeywordProcessor()
    for phrase in vec:
        keyword_processor.add_keyword(phrase)
    found_phrases = keyword_processor.extract_keywords(doc)
    relevance = len(found_phrases)/len(vec)
    missing_phrases = set(vec).difference(set(found_phrases))
    return relevance, missing_phrases

In [161]:
docs[1]

'<h1>Обслуживание кондиционеров</h1> Какой бы ни была цена на сервисное обслуживание кондиционеров, а заниматься поломанными или вышедшими из строя кондиционерами всё равно рано или поздно нужно. Однако если вы дорожите не только своим аппаратом, но и денежными средствами – закажите техническое обслуживание кондиционеров у нас, ведь мы предлагаем самую низкую стоимость на услуги.  Кстати, если вам пока обслуживать по сути ничего, потому что вы так и не приобрели кондиционер, то вам стоит обратить внимание на наш <h href=“на_группу_1”магазин кондиционеров</a>. Вероятно, после покупки и установки вам потребуется масса других услуг по сервису вашего кондиционера. Итак, к списку услуг по обслуживанию кондиционеров в нашем сервисном центре мы относим: ремонт сплит систем, установка новых сплитов, чистка, замена комплектующих. Кроме того, предлагаем диагностику и регулярную замену труб. Если вас интересует, сколько стоит обслуживание именно вашего кондиционера – обратитесь по телефонам, указ

In [155]:
relevance(nvecs[1],ndocs[1])

(0.2,
 {'обслуживание бытовой кондиционер',
  'обслуживание кондиционер daikin',
  'обслуживание кондиционер lg',
  'обслуживание кондиционер mitsubishi',
  'обслуживание кондиционер panasonic',
  'обслуживание кондиционер работа',
  'обслуживание кондиционер сплит система',
  'обслуживание кондиционер цена',
  'обслуживание сплит система',
  'прайс обслуживание кондиционер',
  'разовый обслуживание кондиционер',
  'расценка обслуживание кондиционер',
  'ремонт бытовой кондиционер',
  'ремонт кондиционер цена',
  'ремонт обслуживание кондиционер',
  'ремонт установка кондиционер',
  'сервисный обслуживание кондиционер цена',
  'сколько стоить обслуживание кондиционер',
  'срочный ремонт кондиционер',
  'стоимость обслуживание кондиционер',
  'технический обслуживание кондиционер стоимость',
  'технический обслуживание кондиционер цена',
  'техобслуживание кондиционер',
  'чистка кондиционер'})

### Tf-idf

In [17]:
nvecs = [TextNormalizer.normalize(text) for text in vecs]

In [18]:
ndocs = [TextNormalizer.normalize(doc) for doc in docs]

In [19]:
dct = Dictionary(nvecs)

In [20]:
vec_corpus = [dct.doc2bow(vec) for vec in nvecs]

In [21]:
doc_corpus = [dct.doc2bow(doc) for doc in ndocs]

In [23]:
model = TfidfModel(vec_corpus,dictionary=dct)

In [24]:
tf_vecs = [model[vec] for vec in vec_corpus]

In [25]:
tf_docs = [model[doc] for doc in doc_corpus]

In [26]:
index = similarities.SparseMatrixSimilarity(tf_vecs, num_features=dct.num_pos)

In [36]:
index[tf_docs[3]]

array([0.07533741, 0.06170711, 0.10711007, 0.78122646], dtype=float32)